# Evolutionary algorithms analysis



In [1]:
# Evo algorithms setup

# Imports
import source.visualization as vis
import source.evolutionary_algorithms as ea
import source.benchmark_functions as bf
import numpy as np
import source.helpers as hp
import importlib
import inspect
import sys
import os

# Load benchmark functions
module_path = os.path.abspath(os.path.join('./source'))
sys.path.append(module_path)
benchmarks = importlib.import_module('benchmark_functions')
functions = [obj for name, obj in inspect.getmembers(benchmarks) if inspect.isfunction(obj)]

# Hyperparameters
DIMENSIONS = [2] #, 10, 30]

lower_bound = -100.0
upper_bound = 100.0

POPULATION_SIZES = [10, 20, 50]
FUNCTION_EVALUATIONS = 2000
RESULTS = dict()
REPEATS = 30

## Running ea

Run ea for benchmark functions and write results to global variables.


In [2]:
for i, dim in enumerate(DIMENSIONS):
    BOUNDS = np.array([[lower_bound, upper_bound]] * DIMENSIONS[i])
    pop_size = POPULATION_SIZES[i]
    evaluations = dim * FUNCTION_EVALUATIONS
    migrations = int(np.floor(0.2 * pop_size))
    dimension_results = []
    print(f"Calculating dimension {dim} - ", end="")
    counter = 0

    for function in functions:
        row = [0, 0, 0, 0, 0]
        counter += 1

        for _ in range(REPEATS):
            row[0] += ea.differential_evolution(function, BOUNDS, pop_size, strategy="rand/1/bin")[1]
            row[1] += ea.differential_evolution(function, BOUNDS, pop_size, strategy="best/1/bin")[1]
            row[2] += ea.particle_swarm_optimization(function, BOUNDS, pop_size, evaluations)[1]
            row[3] += ea.soma_ato(function, BOUNDS, pop_size, migrations)[1]
            row[4] += ea.soma_ata(function, BOUNDS, pop_size, migrations)[1]

        print("#", end="")
        if counter % 5 == 0:
            print("|", end="")

        
        row = [x / REPEATS for x in row]
        dimension_results.append(row)
    
    RESULTS[dim] = dimension_results
    print("")

Calculating dimension 2 - #####|#####|#####|#####|#####|


## Analyze results

In [5]:
for dim in RESULTS:
    print(f"\nResults for {dim} dimensions:\n")
    ranking_table = []
    hp.table_header()
    for i, dim_results in enumerate(RESULTS[dim]):
        res = hp.rank_array(dim_results)
        hp.table_row(functions[i]._custom_name, res)
        ranking_table.append(res)
    hp.table_footer(ranking_table)
    chi, p = hp.friedman_test(ranking_table)
    print(f"Chi-square: {chi} | p-value: {p}")
    print("\n\n")


Results for 2 dimensions:

---------------------------------------------------------------------------------------------------------------------------
| Function            |     DE Rand 1     |     DE Best 1     |        PSO        |  SOMA all-to-one  |  SOMA all-to-all  |
---------------------------------------------------------------------------------------------------------------------------
| Ackley 1st          |         1         |         2         |         3         |         5         |         4         |
| Ackley Altered      |         2         |         3         |         1         |         4         |         5         |
| Alpine 1st function |         2         |         3         |         1         |         5         |         4         |
| Alpine 2nd          |         1         |         3         |         2         |         5         |         4         |
| Csendes             |         2         |         3         |         1         |         5         | 